# [모듈 2.2] SageMaker 앤드포인트에 한개의 모델 Triton 배포

# 1. 환경 셋업

## 1.1. 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

전 노트북에서 훈련 후의 아티펙트를 가져옵니다.

In [ ]:
%store -r model_serving_folder
%store -r model_name
%store -r bucket

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

import boto3, json, sagemaker, time
import numpy as np
sm_client = boto3.client(service_name="sagemaker")

## 1.2. 변수 설정

In [ ]:
prefix = "triton-ncf"

ts = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
# endpoint variables
sm_model_name = f"{prefix}-mdl-{ts}"
endpoint_config_name = f"{prefix}-epc-{ts}"
endpoint_name = f"{prefix}-ep-{ts}"
model_data_url = f"s3://{bucket}/{prefix}/"
instance_type = "local_gpu"

In [ ]:
print("sm_model_name: \n", sm_model_name)
print("endpoint_config_name: \n", endpoint_config_name)
print("endpoint_name: \n", endpoint_name)

## 1.3. Triton Docker Image 결정

In [ ]:
from triton_util import account_id_map
region = boto3.Session().region_name

base = "amazonaws.com.cn" if region.startswith("cn-") else "amazonaws.com"
mme_triton_image_uri = (
    "{account_id}.dkr.ecr.{region}.{base}/sagemaker-tritonserver:22.07-py3".format(
        account_id=account_id_map[region], region=region, base=base
    )
)
print("mme_triton_image_uri: \n", mme_triton_image_uri)

In [ ]:
model_serving_folder

# 2. 모델 패키징 (model.tar.gz) 및 S3 업로딩



In [ ]:
import os
from triton_util import tar_artifact, upload_tar_s3

    
model_tar_file = tar_artifact(model_serving_folder, model_name)    
print("model_tar_file: ", model_tar_file)
model_uri_pt = upload_tar_s3(sagemaker_session, model_tar_file, prefix)
print("model_uri_pt: ", model_uri_pt)

# 3. 로컬 모드 설정
- 내부적으로 Triton 서버가 구동시에 아래 URL 스크립트가 구동 됨.
    - 여기에 맞는 필요한 환경 변수를 넣어 줌.
    - https://raw.githubusercontent.com/triton-inference-server/server/main/docker/sagemaker/serve

In [ ]:
from sagemaker.model import Model

In [ ]:
container_envs = {
                    "SAGEMAKER_TRITON_LOG_VERBOSE": "3",
                    "SAGEMAKER_TRITON_LOG_INFO": "1",
                    "SAGEMAKER_TRITON_LOG_WARNING" : "1",
                    "SAGEMAKER_TRITON_LOG_ERROR" : "1"
                 }

local_pytorch_model = Model(model_data= model_uri_pt,
                            image_uri = mme_triton_image_uri,
                            role=role,
                            env = container_envs
                            )


In [ ]:


local_predictor = local_pytorch_model.deploy(
                           instance_type=instance_type, 
                           initial_instance_count=1, 
                           endpoint_name=endpoint_name,
                           wait=True,
                           log = False,
                        )

# 4. 추론 테스트

In [ ]:
def create_sample_payload():
    # user
    user_np = np.zeros((1,100)).astype(np.int32)
    # item
    item_np = np.random.randint(low=1, high=1000, size=(1,100)).astype(np.int32)

    payload = {
        "inputs": [
            {"name": "INPUT__0", "shape": [1,100], 
             "datatype": "INT32", "data": user_np.tolist()},
            {"name": "INPUT__1", "shape": [1,100], 
             "datatype": "INT32", "data": item_np.tolist()},
        ]
    }
    
    return payload

payload = create_sample_payload()
print("payload: ", payload)

In [ ]:
def single_model_invoke_endpoint(client,endpoint_name, payload): 
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/octet-stream", 
        Body=json.dumps(payload),
    )

    result = json.loads(response["Body"].read().decode("utf8"))
    
    return result

runtime_client = sagemaker.local.LocalSagemakerRuntimeClient()    
result = single_model_invoke_endpoint(runtime_client,endpoint_name, payload)
print("result : ", result)

# 5. 로컬 앤드포인트 삭제

In [ ]:
from inference_utils import delete_endpoint

In [ ]:
client = sagemaker.local.LocalSagemakerClient()
delete_endpoint(client, endpoint_name)

# 6. 클라우드 배포

## 6.1. 변수 및 컨테이너 설정

In [ ]:
sm_model_name = f"{prefix}-mdl-{ts}"
real_endpoint_config_name = f"{prefix}-epc-{ts}"
real_endpoint_name = f"{prefix}-ep-{ts}"

In [ ]:
container = {"Image": mme_triton_image_uri,
             "ModelDataUrl": model_uri_pt}


In [ ]:
print("container: ", container)
print("sm_model_name: ", sm_model_name)

## 6.2. 세이지 메이커 모델, 앤드포인트 컨피그, 앤드포인트 생성

In [ ]:
create_model_response = sm_client.create_model(
    ModelName=sm_model_name, ExecutionRoleArn=role, PrimaryContainer=container
)

print("Model Arn: " + create_model_response["ModelArn"])

In [ ]:
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=real_endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.g4dn.4xlarge",
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": sm_model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=real_endpoint_name, EndpointConfigName= real_endpoint_config_name
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

In [ ]:
%%time 

resp = sm_client.describe_endpoint(EndpointName= real_endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=real_endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

## 6.3. 추론

In [ ]:
runtime_client = boto3.Session().client('sagemaker-runtime')
single_model_invoke_endpoint(runtime_client,real_endpoint_name, payload)


# 7. 앤드포인트 삭제

In [ ]:
client = boto3.Session().client('sagemaker')
delete_endpoint(client, real_endpoint_name)